You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!uv tool install crewai crewai_tools langchain_community
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew
from crewai import LLM
from dotenv import load_dotenv
import os

load_dotenv()  # this will load env keys from .env file

True

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see its documentation at https://docs.crewai.com/concepts/llms

In [3]:
api_key = os.getenv('GROQ_API_KEY')
llm = LLM(
    model="groq/gemma2-9b-it",
    temperature = 0.7
)

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=llm
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the 'Content Writer'. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the written blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=False
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
## Content Plan: Artificial Intelligence 

**1. Target Audience:**

* **Primary:**  Tech enthusiasts, business professionals, students interested in STEM fields.
* **Secondary:**  Individuals curious about the impact of AI on everyday life. 

**Pain Points:**

* **Lack of understanding:** Many people are confused about what AI actually is and how it works.
* **Fear of the unknown:** There are concerns about job displacement and the potential misuse of AI.
* **Difficulty keeping up with the rapid advancements:** The field of AI is constantly evolving, making it challenging to stay informed.

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result.raw)

## Demystifying Artificial Intelligence: Trends, Applications, and the Future

Artificial intelligence (AI) is rapidly changing the world, influencing how we work, learn, and interact with technology. It's no longer a futuristic concept limited to science fiction; AI is here, making a tangible impact on our lives. But what exactly is AI, and how is it shaping our future?

**Understanding AI:  More Than Just a Buzzword**

At its core, artificial intelligence refers to the ability of machines to perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making.  Imagine teaching a computer to identify a cat in a picture.  Instead of programming it with specific rules about cat features, you could show it thousands of images labeled "cat." Through machine learning, the computer would identify patterns and features common to cats, eventually learning to recognize them independently.

There are different types of AI, including machine learning and deep learning. Machine learning involves algorithms that learn from data without explicit programming. Deep learning, a subset of machine learning, uses artificial neural networks inspired by the human brain to analyze vast amounts of information.

**The AI Landscape:  Where Innovation is Happening**

The field of AI is constantly evolving, with new breakthroughs and applications emerging at an impressive pace. One of the most exciting trends is **generative AI**, which empowers machines to create new content, such as text, images, and even code. Models like ChatGPT and DALL-E 2 have captured global attention, showcasing the transformative potential of AI in creative industries.

Alongside this progress, there's a growing emphasis on **AI ethics**. As AI systems become more sophisticated, it's crucial to ensure they are developed and deployed responsibly. This involves addressing concerns about bias, fairness, transparency, and accountability, ensuring that AI benefits all of humanity.

**AI in Action:  Real-World Applications**

The applications of AI are vast and continue to expand across various industries:

* **Healthcare:** AI is revolutionizing healthcare by aiding in disease diagnosis, personalizing treatments, and accelerating drug discovery.

* **Finance:** AI-powered algorithms are used for fraud detection, risk assessment, and providing personalized financial advice.

* **Transportation:** Self-driving cars and autonomous delivery robots are poised to reshape how we move goods and people.

These are just a few examples of how AI is transforming industries and shaping our future.

**The Future of AI: A World Shaped by Possibilities**

The potential benefits of AI are immense, but it's essential to approach its development and deployment thoughtfully. As AI becomes more integrated into our lives, we must address ethical challenges and ensure it is used for the betterment of society. The future of AI is not predetermined; it is up to us to shape it in a way that is beneficial and sustainable for all.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "Agenic AI Trend."
result = crew.kickoff(inputs={"topic": topic})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agenic AI Trend..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
## Agenic AI Trend: A Comprehensive Content Plan

**1.  Prioritized Trends, Key Players, and News:**

*   **Trend:**  The rapid development and integration of AI agents capable of autonomous learning, problem-solving, and interaction.

*   **Key Players:** 
    *   **OpenAI:**  Creators of ChatGPT, demonstrating the power of generative AI agents.
    *   **Google DeepMind:**  Developing advanced AI agents like AlphaGo and AlphaFold, showcasing capabilities in game playing and protein folding.
    *   **Anthropic:**  Focuses on building ethical and safe AI agents, emphasizing responsible developme

In [14]:
Markdown(result.raw)

Thought: I now can give a great answer

## Agenic AI:  Intelligent Agents Shaping Our Future 

The world of technology is in constant flux, and few advancements are as profound as the rise of **Agenic AI**. This refers to the development of artificial intelligence agents capable of independent learning, problem-solving, and interaction with their environment. Imagine AI that can not only process information but also learn from it, adapt to new situations, and even make decisions without direct human intervention. This is the promise of Agenic AI, and it's rapidly becoming a reality. 

This shift towards more autonomous and adaptable AI is driven by significant breakthroughs in machine learning, natural language processing, and computer vision. These advancements have enabled AI agents to perform increasingly complex tasks, blurring the lines between human and machine intelligence.

### A Brief History: From Rules to Revolution 

The journey towards Agenic AI has been marked by milestones in artificial intelligence research.  Early AI systems relied on pre-programmed rules, limiting their ability to adapt to new situations. The development of expert systems brought a degree of sophistication, allowing AI to mimic the decision-making of human experts in specific domains. However, it was the advent of deep learning, a powerful subset of machine learning, that truly revolutionized AI. Deep learning algorithms, inspired by the structure of the human brain, can learn complex patterns from vast amounts of data, enabling AI agents to perform tasks like image recognition, speech synthesis, and natural language understanding with unprecedented accuracy.

###  Key Players Shaping the Future

The field of Agenic AI is attracting significant attention from both established tech giants and innovative startups. 

* **OpenAI**, known for its groundbreaking ChatGPT, is at the forefront of developing generative AI agents capable of creating human-quality text, code, and even art.
* **Google DeepMind**, renowned for its breakthroughs in game playing with AlphaGo and protein folding with AlphaFold, is pushing the boundaries of AI capabilities.
* **Anthropic**, a company founded by former OpenAI researchers, focuses on building ethical and safe AI agents, emphasizing responsible development practices.
* **Cohere**, a Canadian AI company, provides tools and APIs for developers to integrate AI agents into their applications, democratizing access to this transformative technology.

These are just a few examples of the companies leading the charge in Agenic AI. Their innovations are constantly shaping the landscape of this rapidly evolving field.

### Transforming Industries, Empowering Individuals

The potential applications of Agenic AI are vast and span across diverse industries:

* **Customer Service:** AI-powered chatbots and virtual assistants are already transforming customer service, providing quick and efficient solutions to common queries, freeing up human agents to handle more complex issues. Imagine a chatbot that can not only answer your questions but also understand your needs and offer personalized solutions.
* **Education:** Personalized learning experiences powered by AI agents can adapt to individual student needs, providing tailored instruction and support. Intelligent tutoring systems can offer real-time feedback and guidance, helping students learn more effectively. 

* **Healthcare:** AI agents can assist in diagnosis, analyze medical images, and even contribute to drug discovery. They can also monitor patient health remotely, providing timely alerts and interventions.
* **Research:** AI agents can analyze massive datasets, identify patterns, and generate hypotheses, accelerating scientific discovery in fields like climate science, genomics, and materials science.

These are just a few examples of how Agenic AI is poised to revolutionize various aspects of our lives.

###  Navigating the Ethical Landscape

While the potential of Agenic AI is immense, it also raises important ethical considerations:

* **Bias and Fairness:** AI agents learn from the data they are trained on, and if that data reflects existing societal biases, the AI can perpetuate those biases. It's crucial to ensure that AI agents are trained on diverse and representative data to mitigate bias.

* **Transparency and Explainability:** Understanding how AI agents make decisions is essential for building trust and accountability. Research into making AI models more transparent and explainable is crucial.
* **Job Displacement:** As AI agents become more capable, there are concerns about potential job displacement. It's important to consider the societal impact of AI and develop strategies for reskilling and upskilling the workforce.

These are complex challenges that require careful consideration and collaboration between researchers, policymakers, and industry leaders.

**A Future Shaped by Intelligent Agents**


Agenic AI is undoubtedly one of the most transformative technologies of our time. Its ability to learn, adapt, and solve problems autonomously has the potential to revolutionize countless industries and aspects of our lives. As this technology continues to evolve, it is essential to navigate the ethical challenges and ensure that AI benefits all of humanity.  




